In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import datasets
from tqdm import tqdm
from sklearn import metrics
from sklearn import model_selection

In [2]:
data = pd.read_csv("../data/cases_train_processed3.csv", parse_dates = True)
model_path = "../models/"

In [3]:
data

,age,sex,province,country,latitude,longitude,outcome,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,35,male,Herat,Afghanistan,34.341944,62.203056,nonhospitalized,38919.0,1437.0,32576.0,4906.0,Afghanistan,99.976005,3.692284
1,45,male,Herat,Afghanistan,34.341944,62.203056,nonhospitalized,38919.0,1437.0,32576.0,4906.0,Afghanistan,99.976005,3.692284
2,52,male,NaN,Albania,41.153030,20.061890,nonhospitalized,12226.0,358.0,6888.0,4980.0,Albania,424.838418,2.928186
3,46,male,NaN,Albania,41.153030,20.061890,nonhospitalized,12226.0,358.0,6888.0,4980.0,Albania,424.838418,2.928186
4,34,male,NaN,Albania,41.153030,20.061890,nonhospitalized,12226.0,358.0,6888.0,4980.0,Albania,424.838418,2.928186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345466,46,female,Mash East,Zimbabwe,-18.321880,30.594450,recovered,7672.0,225.0,5914.0,1533.0,Zimbabwe,51.618366,2.932742
345467,52,female,Mash East,Zimbabwe,-18.321880,30.594450,deceased,7672.0,225.0,5914.0,1533.0,Zimbabwe,51.618366,2.932742
345468,42,male,Mash West,Zimbabwe,-17.376550,30.192260,recovered,7672.0,225.0,5914.0,1533.0,Zimbabwe,51.618366,2.932742
345469,30,male,Mash West,Zimbabwe,-18.129860,30.150540,recovered,7672.0,225.0,5914.0,1533.0,Zimbabwe,51.618366,2.932742


In [4]:
test_data = pd.read_csv("../data/cases_test_processed.csv", parse_dates = True)
#model_path = "../models/"

In [5]:
test_data

,age,sex,province,country,latitude,longitude,outcome,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,15,male,NaN,Albania,41.15303,20.061890,NaN,12226.0,358.0,6888.0,4980.0,Albania,424.838418,2.928186
1,42,male,Ain Defla,Algeria,36.26520,1.970300,NaN,49623.0,1665.0,34923.0,13035.0,Algeria,113.162645,3.355299
2,45,male,Annaba,Algeria,36.90000,7.766667,NaN,49623.0,1665.0,34923.0,13035.0,Algeria,113.162645,3.355299
3,32,male,Bejaia,Algeria,36.75000,5.066667,NaN,49623.0,1665.0,34923.0,13035.0,Algeria,113.162645,3.355299
4,48,male,Bejaia,Algeria,36.75000,5.066667,NaN,49623.0,1665.0,34923.0,13035.0,Algeria,113.162645,3.355299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46495,52,female,Bulawayo,Zimbabwe,-20.17000,28.580000,NaN,7672.0,225.0,5914.0,1533.0,Zimbabwe,51.618366,2.932742
46496,27,male,Bulawayo Metro,Zimbabwe,-20.17608,28.573630,NaN,7672.0,225.0,5914.0,1533.0,Zimbabwe,51.618366,2.932742
46497,40,female,Harare,Zimbabwe,-17.85000,31.050000,NaN,7672.0,225.0,5914.0,1533.0,Zimbabwe,51.618366,2.932742
46498,24,female,Harare,Zimbabwe,-17.85000,31.050000,NaN,7672.0,225.0,5914.0,1533.0,Zimbabwe,51.618366,2.932742


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 345471 entries, 0 to 345470
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   age                  345471 non-null  int64  
 1   sex                  345471 non-null  object 
 2   province             341444 non-null  object 
 3   country              345471 non-null  object 
 4   latitude             345471 non-null  float64
 5   longitude            345471 non-null  float64
 6   outcome              345471 non-null  object 
 7   Confirmed            345471 non-null  float64
 8   Deaths               345471 non-null  float64
 9   Recovered            345471 non-null  float64
 10  Active               345471 non-null  float64
 11  Combined_Key         345471 non-null  object 
 12  Incidence_Rate       345471 non-null  float64
 13  Case-Fatality_Ratio  345471 non-null  float64
dtypes: float64(8), int64(1), object(5)
memory usage: 36.9+ MB


In [7]:
data.isnull().sum()

age                       0
sex                       0
province               4027
country                   0
latitude                  0
longitude                 0
outcome                   0
Confirmed                 0
Deaths                    0
Recovered                 0
Active                    0
Combined_Key              0
Incidence_Rate            0
Case-Fatality_Ratio       0
dtype: int64

In [8]:
def categorize_column(data):
    i = 0
    for value in tqdm(data.unique()):
        data.replace(value, i, inplace = True)
        i += 1
    data = data.apply(pd.to_numeric)
    return data

In [9]:
def categorize_outcome(data):
    data = data.map({'nonhospitalized':0, 'deceased':1, 'recovered':2, 'hospitalized':3},na_action ='ignore')
    data = data.apply(pd.to_numeric)
    return data

In [10]:
data['sex'] = categorize_column(data['sex'])
data['outcome'] = categorize_outcome(data['outcome'])
data['Combined_Key'] = categorize_column(data['Combined_Key'])

100%|██████████| 489/489 [00:22<00:00, 22.10it/s] 


In [11]:
#data = data.loc[~data['date_confirmation'].str.contains('-'), :]

In [12]:
data = data.drop(columns=['province', 'country'])
#data = data.drop(columns=['date_confirmation'])

In [13]:
X = data.drop(columns=['outcome'])
y = data['outcome']

In [14]:
X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size=0.2, random_state=69, shuffle=True)
#X_train, X_valid, y_train, y_valid = train_test_split(X,y)

In [15]:
knn_model = KNeighborsClassifier(n_neighbors=100,p=2,leaf_size=30,weights='uniform')

In [16]:
knn_model.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=100)

In [17]:
print("knn model valid score:", knn_model.score(X_valid, y_valid) * 100.0)
#print('%\n')
print("knn model train score:", knn_model.score(X_train, y_train) * 100.0)
#print('%\n')

knn model valid score: 73.64498154714524
knn model train score: 74.36173908009378


In [18]:
y_pred = knn_model.predict(X_valid)

In [19]:
#y_pred = metrics.log_reg.predict(X_valid)

In [20]:
print(metrics.classification_report(y_valid, y_pred, zero_division=1))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98     26278
           1       1.00      0.00      0.00       921
           2       0.52      0.25      0.34     17647
           3       0.61      0.84      0.70     24249

    accuracy                           0.74     69095
   macro avg       0.77      0.52      0.51     69095
weighted avg       0.73      0.74      0.71     69095



In [21]:
#metrics.roc_auc_score(y_valid, y_pred, multi_class='ovr')

In [22]:
#hyperparameters testing

In [23]:
#List Hyperparameters that we want to tune.
leaf_size = list(range(1,30))
n_neighbors = list(range(1,20))
p=[1,2]#Convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)#Create new KNN object


In [24]:
knn_2 = KNeighborsClassifier()#Use GridSearch
clf = model_selection.GridSearchCV(knn_2, hyperparameters, cv=10)#Fit the model


In [ ]:
best_model = clf.fit(X,y)#Print The value of best Hyperparameters
print('Best leaf_size:', best_model.best_estimator_.get_params()['leaf_size'])
print('Best p:', best_model.best_estimator_.get_params()['p'])
print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])